# Text Classification
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. Please check the pdf file for more details.*

In this exercise you will:
    
- implement a of spam classifier with **Naive Bayes method** for real world email messages
- learn the **training and testing phase** for Naive Bayes classifier  
- get an idea of the **precision-recall** tradeoff

In [1]:
# some basic imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
# ham_train contains the occurrences of each word in ham emails. 1-by-N vector
ham_train = np.loadtxt('ham_train.csv', delimiter=',')
# spam_train contains the occurrences of each word in spam emails. 1-by-N vector
spam_train = np.loadtxt('spam_train.csv', delimiter=',')
# N is the size of vocabulary.
N = ham_train.shape[0]
# There 9034 ham emails and 3372 spam emails in the training samples
num_ham_train = 9034
num_spam_train = 3372
# Do smoothing
x = np.vstack([ham_train, spam_train]) + 1

# ham_test contains the occurences of each word in each ham test email. P-by-N vector, with P is number of ham test emails.
i,j,ham_test = np.loadtxt('ham_test.txt').T
i = i.astype(np.int)
j = j.astype(np.int)
#第i个邮件词库第j个单词的位置
ham_test_tight = scipy.sparse.coo_matrix((ham_test, (i - 1, j - 1)))
ham_test = scipy.sparse.csr_matrix((ham_test_tight.shape[0], ham_train.shape[0]))
ham_test[:, 0:ham_test_tight.shape[1]] = ham_test_tight
# spam_test contains the occurences of each word in each spam test email. Q-by-N vector, with Q is number of spam test emails.
i,j,spam_test = np.loadtxt('spam_test.txt').T
i = i.astype(np.int)
j = j.astype(np.int)
spam_test_tight = scipy.sparse.csr_matrix((spam_test, (i - 1, j - 1)))
spam_test = scipy.sparse.csr_matrix((spam_test_tight.shape[0], spam_train.shape[0]))
spam_test[:, 0:spam_test_tight.shape[1]] = spam_test_tight


D:\Anaconda\lib\site-packages\scipy\sparse\_index.py:118: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


## Now let's implement a ham/spam email classifier. Please refer to the PDF file for details

In [3]:
from likelihood import likelihood
import re
# TODO
# Implement a ham/spam email classifier, and calculate the accuracy of your classifier

# Hint: you can directly do matrix multiply between scipy.sparse.coo_matrix and numpy.array.
# Specifically, you can use sparse_matrix * np_array to do this. Note that when you use "*" operator
# between numpy array, this is typically an elementwise multiply.

# begin answer
l=likelihood(x)
rate=np.zeros(x.shape[0])
rate=l[0]/l[1]
top10_index = np.argpartition(rate, range(10))[:10]

f = open("all_word_map.txt", "r", encoding='utf-8')
dic = {}
for line in f.readlines():
    word, index = re.split(re.compile(r'\s+'), line.strip())
    dic[int(index)] = word

for index in top10_index:
    print(dic[index+1])
# end answer

nbsp
viagra
pills
cialis
voip
php
meds
computron
sex
ooking


In [10]:
# prior
prior = np.log(np.array([num_ham_train, num_spam_train])/(num_ham_train + num_spam_train))

#likelihood
ham_likelihood = np.log(l[0]).reshape((-1,1))
spam_likelihood = np.log(l[1]).reshape((-1,1))

# posterior
ham_post1 = np.matmul(ham_test.todense(), ham_likelihood) + prior[0]
ham_post2 = np.matmul(ham_test.todense(), spam_likelihood) + prior[1]
spam_post1 = np.matmul(spam_test.todense(), spam_likelihood) + prior[1]
spam_post2 = np.matmul(spam_test.todense(), ham_likelihood) + prior[0]

#accuracy
correct_ham = np.sum(ham_post1 > ham_post2)
correct_spam = np.sum(spam_post1 > spam_post2)
accuracy = (correct_ham + correct_spam) / (ham_test.shape[0] + spam_test.shape[0])
print(accuracy)

#precision&recall
precision = correct_spam / (correct_spam + ham_test.shape[0] - correct_ham)
recall = correct_spam / spam_test.shape[0]

print(precision, recall)

0.9857315598548972
0.9750223015165032 0.9724199288256228
